In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/validation/val_data1.json
/kaggle/input/trainnn/train_data1.json


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import json
import numpy as np
import re
import string
import pandas as pd
import matplotlib.pyplot as plt
from torch.nn import Transformer
import math

!pip install nltk
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
with open('/kaggle/input/trainnn/train_data1.json', 'r') as file: # Replace this path with the dataset path in your local machine
    data = json.load(file)

In [4]:
# Process JSON data
source_sentences_train = []
target_sentences_train = []

source_sentences_val = []
target_sentences_val = []

id_train = []
id_val = []

In [5]:
for language_pair, language_data in data.items():
  print(f"Language Pair: {language_pair}")


Language Pair: English-Bengali
Language Pair: English-Hindi


In [6]:
for language_pair, language_data in data.items():
    if(language_pair == "English-Hindi"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"]
              target = entry_data["target"]
              if (data_type == "Validation"):
                source_sentences_val.append(source)
                target_sentences_val.append(target)
                id_val.append(entry_id)
              else:
                source_sentences_train.append(source)
                target_sentences_train.append(target)
                id_train.append(entry_id)

Language Pair: English-Hindi
  Data Type: Train


In [7]:
with open('/kaggle/input/validation/val_data1.json', 'r') as file: # Replace this path with the dataset path in your local machine
    data = json.load(file)

In [8]:
for language_pair, language_data in data.items():
    if(language_pair == "English-Hindi"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"]
              #target = entry_data["target"]
              if (data_type == "Validation"):
                source_sentences_val.append(source)
                #target_sentences_val.append(target)
                #id_val.append(entry_id)
              #else:
                #source_sentences_train.append(source)
                #target_sentences_train.append(target)
                #id_train.append(entry_id)

Language Pair: English-Hindi
  Data Type: Validation


In [9]:
print(len(source_sentences_train))
print(len(target_sentences_train))

print(len(source_sentences_val))
print(len(target_sentences_val))

80797
80797
11543
0


In [10]:
x={'English':source_sentences_train,'Hindi':target_sentences_train}
df=pd.DataFrame(x)
df

,English,Hindi
0,cancel everything on my calendar,मेरे कैलेंडर पर सब कुछ रद्द करें
1,Adrenal hormone levels are at their peak durin...,अधिवृक्क के हार्मोन का स्तर प्रातःकाल में अपने...
2,"Golden threads are obtained from Surat, the qu...","स्वर्ण धागे सूरत से प्राप्त होते हैं, जिनकी गु..."
3,Look for agglutination within 30 seconds.,30 सेकेण्ड के भीतर एग्लूटिनेशन देखें।
4,The non-pompousness and informality of their l...,उनके जीवन की आडंबरहीनता एवं अनौपचारिकता उनके स...
...,...,...
80792,"So, is it that this is the optimization proble...","तो, यह अनुकूलन समस्या है जिसमें हम रुचि रखते थे।"
80793,In this Masjid made with red stones there are ...,लाल पत्थरों से बनायी गयी इस मस्जिद में हिन्दू ...
80794,"He began to work on the movie on August 17, 20...","उन्होंने 17 अगस्त, 2010 को फिल्म पर काम करना श..."
80795,start a new shopping list,एक नई खरीदारी सूची शुरू करें


In [11]:
# Install Indic NLP Library
!pip install indic-nlp-library

# Install Indic NLP Resources (corpus and models)
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
import os
os.environ["INDIC_RESOURCES_PATH"] = "/kaggle/working/indic_nlp_resources"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.9 MB/s eta 0:00:00
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126 (from 1)
Receiving objects: 100% (139/139), 149.77 MiB | 43.49 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [12]:
from collections import Counter
from indicnlp.tokenize import indic_tokenize
import nltk
import re
import string
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory

# Tokenization and Lowercasing
def preprocess_english(sentences):
    tokenized = []
    for sent in sentences:
        # Tokenize first, then process tokens
        tokens = nltk.word_tokenize(sent.lower())
        # Filter punctuation/digits after tokenization
        cleaned = [t for t in tokens if t not in string.punctuation and not t.isdigit()]
        tokenized.append(cleaned)
    return tokenized

factory = IndicNormalizerFactory()
normalizer = factory.get_normalizer("hi")

def preprocess_hindi(sentences):
    processed = []
    hin_punct = re.compile(r'[!\"#\$%&\'\(\)\*\+,\-\./:;<=>\?@\[\\\]\^_`\{\|\}~।॥॰‘’“”…–—]')
    
    for sent in sentences:
        # Normalize variations (e.g., nuqta, vowel signs)
        normalized = normalizer.normalize(sent)
        cleaned = hin_punct.sub(' ', normalized)
        tokens = [t.strip() for t in indic_tokenize.trivial_tokenize(cleaned) if t.strip()]
        processed.append(tokens)
    
    return processed

# Bengali preprocessing
factory_bn = IndicNormalizerFactory()
normalizer_bn = factory_bn.get_normalizer("bn")

def preprocess_bengali(sentences):
    processed = []
    bn_punct = re.compile(r'[!\"#\$%&\'\(\)\*\+,\-\./:;<=>\?@\[\\\]\^_`\{\|\}~।‘’“”…–—৷৹৺]')
    
    for sent in sentences:
        # Normalize Bengali variants
        normalized = normalizer_bn.normalize(sent)
        # Remove punctuation
        cleaned = bn_punct.sub(' ', normalized)
        # Tokenize with IndicNLP
        tokens = [t.strip() for t in indic_tokenize.trivial_tokenize(cleaned) if t.strip()]
        processed.append(tokens)
    
    return processed

In [13]:
# Check alignment between source and target
assert len(source_sentences_train) == len(target_sentences_train), "Mismatched train data!"

In [14]:
target_sentences_train = [re.sub(r'[a-zA-Z]','',hi) for hi in target_sentences_train] #optional

english_tokens = preprocess_english(source_sentences_train)
english_test=preprocess_english(source_sentences_val)
hindi_tokens = preprocess_hindi(target_sentences_train)
hindi_test=preprocess_hindi(target_sentences_val)

en_train=english_tokens
en_test=english_test
de_train=hindi_tokens
de_test=hindi_test

# Print first 3 preprocessed sentences (English and Hindi)
print("=== Preprocessed Sentences ===")
for i in range(3):
    print(f"\nSample {i+1}:")
    print("Original English:", source_sentences_train[i])
    print("Processed English:", en_train[i])
    print("Original Hindi:", target_sentences_train[i])
    print("Processed Hindi:", de_train[i])
    print("-----------------------------")

=== Preprocessed Sentences ===

Sample 1:
Original English: cancel everything on my calendar
Processed English: ['cancel', 'everything', 'on', 'my', 'calendar']
Original Hindi: मेरे कैलेंडर पर सब कुछ रद्द करें
Processed Hindi: ['मेरे', 'कैलेंडर', 'पर', 'सब', 'कुछ', 'रद्द', 'करें']
-----------------------------

Sample 2:
Original English: Adrenal hormone levels are at their peak during the morning and taper off during the evening, reaching a low level around 3 am An important function of these hormones is to regulate vascular muscle tone and to prevent vasocodilation.
Processed English: ['adrenal', 'hormone', 'levels', 'are', 'at', 'their', 'peak', 'during', 'the', 'morning', 'and', 'taper', 'off', 'during', 'the', 'evening', 'reaching', 'a', 'low', 'level', 'around', 'am', 'an', 'important', 'function', 'of', 'these', 'hormones', 'is', 'to', 'regulate', 'vascular', 'muscle', 'tone', 'and', 'to', 'prevent', 'vasocodilation']
Original Hindi: अधिवृक्क के हार्मोन का स्तर प्रातःकाल में अपन

In [15]:
from collections import Counter

def build_vocab(tokenized_sentences, max_vocab_size=60000):
    # Count all tokens across sentences
    word_counts = Counter([word for sent in tokenized_sentences for word in sent])
    # Create vocab with special tokens + top-N frequent words
    vocab = ["<PAD>", "<SOS>", "<EOS>", "<UNK>"] + [word for word, _ in word_counts.most_common(max_vocab_size)]
    return vocab

# Build vocabularies for English and Hindi
en_index2word = build_vocab(en_train)
de_index2word = build_vocab(de_train)

In [16]:
print(en_index2word[:5])

['<PAD>', '<SOS>', '<EOS>', '<UNK>', 'the']


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [18]:
en_word2index = {token: idx for idx, token in enumerate(en_index2word)}
de_word2index = {token: idx for idx, token in enumerate(de_index2word)}

print("en_word2index length", len(en_word2index))

en_lengths = sum([len(sent) for sent in en_train])/len(en_train)
de_lengths = sum([len(sent) for sent in de_train])/len(de_train)
print("en_lengths:", en_lengths)
print("de_lengths:", de_lengths)

print("Max English length:", max(len(s) for s in en_train))
print("Max Hindi length:", max(len(s) for s in de_train))
# Adjust seq_length accordingly

en_word2index length 60004
en_lengths: 16.419297746203448
de_lengths: 18.210267707959453
Max English length: 255
Max Hindi length: 219


In [19]:
seq_length = 100
hidden_size = 256
epochs = 39

def encode_and_pad(vocab, sent, max_length):
    sos = [vocab["<SOS>"]]
    eos = [vocab["<EOS>"]]
    pad = [vocab["<PAD>"]]
    unk = vocab["<UNK>"]

    if len(sent) < max_length - 2:
        n_pads = max_length - 2 - len(sent)
        encoded = [vocab.get(word, unk) for word in sent]  # Use .get() for UNK
        return sos + encoded + eos + pad * n_pads
    else:
        encoded = [vocab.get(word, unk) for word in sent[:max_length-2]]  # Use .get()
        return sos + encoded + eos

In [20]:
en_train_encoded = [encode_and_pad(en_word2index, sent, seq_length) for sent in en_train]
en_test_encoded = [encode_and_pad(en_word2index, sent, seq_length) for sent in en_test]
de_train_encoded = [encode_and_pad(de_word2index, sent, seq_length) for sent in de_train]
de_test_encoded = [encode_and_pad(de_word2index, sent, seq_length) for sent in de_test]

In [21]:
en_train_encoded[1]

[1,
 8308,
 4548,
 1432,
 13,
 25,
 60,
 1242,
 80,
 4,
 494,
 6,
 29732,
 296,
 80,
 4,
 891,
 1385,
 10,
 504,
 233,
 169,
 143,
 36,
 132,
 216,
 5,
 49,
 5417,
 9,
 8,
 9408,
 7505,
 3283,
 4952,
 6,
 8,
 853,
 29733,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [22]:
batch_size = 50
train_x = np.array(en_train_encoded)
train_y = np.array(de_train_encoded)
test_x = np.array(en_test_encoded)
test_y = np.array(de_test_encoded)

train_ds = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
test_ds = TensorDataset(torch.from_numpy(test_x))


train_dl = DataLoader(train_ds, shuffle=True, batch_size=batch_size, drop_last=True)
#test_dl = DataLoader(test_ds, shuffle=True, batch_size=batch_size, drop_last=True)

In [23]:
train_x[1]

array([    1,  8308,  4548,  1432,    13,    25,    60,  1242,    80,
           4,   494,     6, 29732,   296,    80,     4,   891,  1385,
          10,   504,   233,   169,   143,    36,   132,   216,     5,
          49,  5417,     9,     8,  9408,  7505,  3283,  4952,     6,
           8,   853, 29733,     2,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])

In [24]:
train_ds[1]

(tensor([    1,  8308,  4548,  1432,    13,    25,    60,  1242,    80,     4,
           494,     6, 29732,   296,    80,     4,   891,  1385,    10,   504,
           233,   169,   143,    36,   132,   216,     5,    49,  5417,     9,
             8,  9408,  7505,  3283,  4952,     6,     8,   853, 29733,     2,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 tensor([    1,  9753,     4,  4067,    12,   257, 11367,     6,    45,  1626,
            15,    40,     5,     9,   138,   988,  3295,   355,    13,  1464,
           257,    15,  9754,    73, 23056,     6,

In [25]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=500):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, nhead=8, num_encoder_layers=6, 
                 num_decoder_layers=6, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.embed_src = nn.Embedding(src_vocab_size, d_model)
        self.embed_tgt = nn.Embedding(tgt_vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        self.pos_decoder = PositionalEncoding(d_model, dropout)
        
        self.transformer = Transformer(
            d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers, dim_feedforward=dim_feedforward,
            dropout=dropout, batch_first=True
        )
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None, memory_mask=None,
                src_key_padding_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        src = self.embed_src(src) * math.sqrt(self.d_model)
        tgt = self.embed_tgt(tgt) * math.sqrt(self.d_model)
        
        src = self.pos_encoder(src)
        tgt = self.pos_decoder(tgt)
        
        output = self.transformer(
            src, tgt, 
            src_mask=src_mask, tgt_mask=tgt_mask, memory_mask=memory_mask,
            src_key_padding_mask=src_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
            memory_key_padding_mask=memory_key_padding_mask
        )
        return self.fc_out(output)

In [26]:
model = TransformerModel(
    len(en_index2word),
    len(de_index2word),
    d_model=256,
    nhead=8,
    num_encoder_layers=4,
    num_decoder_layers=4,
    dim_feedforward=1024,
    dropout=0.1
).to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Update training loop
def generate_square_subsequent_mask(sz):
    return torch.triu(torch.ones((sz, sz), dtype=torch.bool), diagonal=1)

In [27]:
input_length = target_length = seq_length

SOS = en_word2index["<SOS>"]
EOS = en_word2index["<EOS>"]


# Update the training loop 
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_dl:
        src = batch[0].to(device)
        tgt = batch[1].to(device)
        
        tgt_input = tgt[:, :-1]
        
        src_mask = generate_square_subsequent_mask(src.size(1)).to(device)
        tgt_mask = generate_square_subsequent_mask(tgt_input.size(1)).to(device)
        
        src_padding_mask = (src == 0).to(device)
        tgt_padding_mask = (tgt_input == 0).to(device)
        
        logits = model(
            src, tgt_input,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            src_key_padding_mask=src_padding_mask,
            tgt_key_padding_mask=tgt_padding_mask
        )
        
        # Compute loss and backpropagate
        optimizer.zero_grad()
        loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt[:, 1:].reshape(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f'Epoch {epoch} | Loss: {total_loss/len(train_dl):.4f}')

Epoch 0 | Loss: 2.5382
Epoch 1 | Loss: 2.3447
Epoch 2 | Loss: 2.2533
Epoch 3 | Loss: 2.1812
Epoch 4 | Loss: 2.1208
Epoch 5 | Loss: 2.0695
Epoch 6 | Loss: 2.0258
Epoch 7 | Loss: 1.9882
Epoch 8 | Loss: 1.9554
Epoch 9 | Loss: 1.9275
Epoch 10 | Loss: 1.9026
Epoch 11 | Loss: 1.8816
Epoch 12 | Loss: 1.8631
Epoch 13 | Loss: 1.8470
Epoch 14 | Loss: 1.8338
Epoch 15 | Loss: 1.8216
Epoch 16 | Loss: 1.8109
Epoch 17 | Loss: 1.8007
Epoch 18 | Loss: 1.7920
Epoch 19 | Loss: 1.7839
Epoch 20 | Loss: 1.7761
Epoch 21 | Loss: 1.7694
Epoch 22 | Loss: 1.7632
Epoch 23 | Loss: 1.7572
Epoch 24 | Loss: 1.7519
Epoch 25 | Loss: 1.7464
Epoch 26 | Loss: 1.7416
Epoch 27 | Loss: 1.7366
Epoch 28 | Loss: 1.7325
Epoch 29 | Loss: 1.7280
Epoch 30 | Loss: 1.7243
Epoch 31 | Loss: 1.7202
Epoch 32 | Loss: 1.7169
Epoch 33 | Loss: 1.7134
Epoch 34 | Loss: 1.7100
Epoch 35 | Loss: 1.7070
Epoch 36 | Loss: 1.7037
Epoch 37 | Loss: 1.7007
Epoch 38 | Loss: 1.6980


In [28]:
from tqdm import tqdm
val_ids = [ i for i,_ in data["English-Hindi"]["Validation"].items()]

In [29]:
def generate_translation(model, src, max_length=50, beam_size=5):
    model.eval()
    src = src.unsqueeze(0).to(device)  # Shape: [1, seq_len]
    src_seq_len = src.size(1)
    
    # Encode source sequence
    memory = model.transformer.encoder(
        model.pos_encoder(model.embed_src(src) * math.sqrt(model.d_model)),
        mask=generate_square_subsequent_mask(src_seq_len).to(device),
        src_key_padding_mask=(src == 0).to(device)
    )
    
    # Initialize beam search
    candidates = [([de_word2index["<SOS>"]], 0.0)]
    
    for _ in range(max_length):
        new_candidates = []
        for seq, score in candidates:
            if seq[-1] == de_word2index["<EOS>"]:
                new_candidates.append((seq, score))
                continue
                
            tgt = torch.tensor(seq, device=device).unsqueeze(0)  # Shape: [1, current_seq_len]
            output = model.transformer.decoder(
                model.pos_decoder(model.embed_tgt(tgt) * math.sqrt(model.d_model)),
                memory,
                tgt_mask=generate_square_subsequent_mask(tgt.size(1)).to(device),
                memory_key_padding_mask=(src == 0).to(device)
            )
            
            logits = model.fc_out(output[:, -1, :])
            log_probs = F.log_softmax(logits, dim=-1)
            
            topk_scores, topk_ids = log_probs.topk(beam_size)
            
            for i in range(beam_size):
                new_seq = seq + [topk_ids[0][i].item()]
                new_score = score + topk_scores[0][i].item()
                new_candidates.append((new_seq, new_score))
                
        candidates = sorted(new_candidates, key=lambda x: x[1], reverse=True)[:beam_size]
    
    best_seq = candidates[0][0][1:-1] if len(candidates[0][0]) > 2 else []
    translated_tokens = [de_index2word[idx] for idx in best_seq]
    return ' '.join(translated_tokens)

# Validation loop (unchanged)
val_outs = []
for i in tqdm(range(len(test_ds))):
    src_seq = test_ds[i][0].to(device)
    translated = generate_translation(model, src_seq)
    val_outs.append(translated)

100%|██████████| 11543/11543 [5:02:53<00:00,  1.57s/it]


In [30]:
df0 = pd.DataFrame()
df0["ID"] = val_ids
df0["Translation"] = val_outs

df0.to_csv('/kaggle/working/answersH.csv', index = False)
x=pd.read_csv("/kaggle/working/answersH.csv")
x

,ID,Translation
0,505511,इस ओर किसी तरफ से किसी किसी भी इस ओर किसी तरफ ...
1,505512,अल्कोहल अल्कोहल अल्कोहल अल्कोहल अल्कोहल अल्कोह...
2,505513,जौ जौ का उपयोग भी माल्ट उत्पादन के लिए भी किया...
3,505514,<UNK> <UNK> ने इस फिल्म के लिए <UNK> <UNK> <UN...
4,505515,<UNK> <UNK> <UNK> <UNK> नेशनल पार्क वर्ल्ड <UN...
...,...,...
11538,517049,अपने पुत्र भोज के साथ काँच के पुत्र भोज और उनक...
11539,517050,यह मुख्य रूप से उन क्षेत्रों में होता है जो मु...
11540,517051,मुझे हिमाचल प्रदेश हिमाचल प्रदेश हिमाचल प्रदेश...
11541,517052,प्राग में लोग चाहेंगे जाने वाले लोगों के मौसम ...
